In [ ]:
!pip install pinecone transformers sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [ ]:
!pip install transformers sentence-transformers


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.5 MB/s eta 0:00:00


In [ ]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 95.6 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for i, page in enumerate(doc):
        text += f"\n--- Page {i+1} ---\n"
        text += page.get_text()
    return text

text_output = extract_text_from_pdf("MIMS.pdf")
print(text_output[:2000])


In [ ]:
def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(text_output)


In [ ]:
MAX_BYTES = 4 * 1024 * 1024  # 4MB
def split_payload(payload_list, encoder_fn):
    current_batch = []
    current_size = 0
    for item in payload_list:
        encoded = encoder_fn(item)  # e.g., JSON-encoded string or vector
        size = len(encoded.encode("utf-8"))
        if current_size + size > MAX_BYTES:
            yield current_batch
            current_batch = [item]
            current_size = size
        else:
            current_batch.append(item)
            current_size += size
    if current_batch:
        yield current_batch


In [ ]:
from sentence_transformers import SentenceTransformer
import pinecone

# Chunk text (assuming this is already done and stored in 'chunks')
# chunks = chunk_text(text_output)

# Load SentenceTransformer model
embedder = SentenceTransformer("BAAI/bge-large-en", device='cuda')

#  Ensure chunks is a list of strings
texts = [chunk['text'] if isinstance(chunk, dict) else chunk for chunk in chunks]

# Embed the chunks
embeddings = embedder.encode(texts, show_progress_bar=True, batch_size=16)

# Initialize Pinecone client
pc = Pinecone(api_key="pcsk_4Xke5d_QK4YNgeake3By84gyFxiRRVZP2vk7riRL5jXciZH47RtoYJep584XqaFCJaFoBZ", environment="us-east-1")


# Connect to index
index = pc.Index("medicalbooks")

# Prepare vectors for upsert
vectors = [
    (f"chunk-{i}", embeddings[i].tolist(), {"text": texts[i]})
    for i in range(len(texts))
]

# Batch upsert (recommended by Pinecone to avoid size limit errors)
batch_size = 100
for i in range(0, len(vectors), batch_size):
    batch = vectors[i:i + batch_size]
    index.upsert(vectors=batch)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

NameError: name 'chunks' is not defined

In [ ]:
from IPython.display import HTML, display

# Enable text wrapping in all outputs
display(HTML('''
<style>
.output pre {
    white-space: pre-wrap;   /* Wrap long lines */
    word-wrap: break-word;   /* Break very long words if needed */
}
</style>
'''))


### To test run the following and change the question in the end

In [1]:
!pip install pinecone transformers sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.0 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("BAAI/bge-large-en", device='cuda')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [10]:
import pinecone
pc = pinecone.Pinecone(api_key="pcsk_5w8bJf_Umqqm1NKSfDcmrSdSApd2qyoNmSGQQXt34XKFYqQBJNnNWDbD2VT8gc19kwK53c", environment="us-east-1")
index = pc.Index("medicalbooks")

In [17]:
def retrieve_context(query, top_k=10):
    query_vec = embedder.encode([query])[0].tolist()
    results = index.query(vector=query_vec, top_k=top_k, include_metadata=True)
    return [match["metadata"]["text"] for match in results["matches"]]


In [18]:
from openai import OpenAI

client = OpenAI(
    api_key="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",
    base_url="https://api.groq.com/openai/v1"
)


def generate_rag_response(user_query):
    context_chunks = retrieve_context(user_query)
    context = "\n".join(context_chunks)

    prompt = f"""
You are a medical assistant RAG system. You must reason step-by-step.
Provide your response in this format:

THOUGHT: <detailed reasoning>
FINAL ANSWER: <concise answer to the user>

Use ONLY the context below:
{context}

User Question: {user_query}
    """
    response = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content


In [19]:
question = "what percentage of patients with T2DM will eventually die from CV complications?"
response = generate_rag_response(question)
print("Bot:", response)


Bot: <think>
Okay, I'm trying to figure out what percentage of patients with Type 2 Diabetes Mellitus (T2DM) will eventually die from cardiovascular (CV) complications. Let me start by recalling what I know about T2DM and its complications.

I remember that T2DM is the more common form of diabetes, accounting for about 90-95% of all diabetes cases. It's associated with insulin resistance and often obesity. The context provided mentions that T2DM can lead to complications like coronary heart disease, cerebrovascular disease, retinopathy, nephropathy, and neuropathy. These complications can significantly impact a patient's quality of life and life expectancy.

Looking through the context, I see that coronary heart disease can lead to heart attacks, and cerebrovascular disease can cause strokes. Both of these are major CV complications. The context also talks about the importance of managing factors like hypertension and hyperlipidemia, which are known risk factors for CV diseases.

I rec